# Conversation Summary
---

Veamos otro tipo de memoria, esta es un poco mas compleja, `ConversationSummaryMemory`. Este tipo de memoria crea un resumen de la conversacion a lo largo del tiempo.

Esto puede resultar util para condensar la informacion de la conversacion. La memoria va siendo resumida a memdida que ocurre y almacena el resumen actual en la memoria.

Esta memoria se puede utilizar para inyectar el resumen de la conversacion hasta el momento en un mensaje.

Este tipo de memoria es mas util para conversaciones mas largas donde mantener el historial de mensajes pasados en el mensaje palabra por palabra consumiria demasiados tokkens.



### Funcionalidad basica
---
Hagamos unos ejemplos con su forma mas basica

In [2]:
from dotenv import load_dotenv, find_dotenv
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.llms import OpenAI

load_dotenv(find_dotenv())

True

Realizemos el primer ejemplo, instanciando la memoria

In [3]:
# llm
llm = OpenAI(temperature=0)

# memory
memory = ConversationSummaryMemory(
    llm=llm
)

# Add context to memory
memory.save_context(
    inputs={"input": "hi"},
    outputs={"output": "whats up"}
)

Para mostrar el contenido de la `memoria` usaremos el metodo `load_memory_variables`

In [4]:
memory.load_memory_variables({})

{'history': '\nThe human greets the AI and the AI responds with a casual greeting.'}

Podemos obtener el historial como una lista de mensajes (recordemos que esto es util para cuando usamos un modelo de "chat")

In [6]:
# llm
llm = OpenAI(temperature=0)

# memory
memory = ConversationSummaryMemory(
    llm=llm,
    return_messages=True,
)

# Add context to memory
memory.save_context(
    inputs={"input": "hi"},
    outputs={"output": "whats up"}
)

De igual forma, obtenemos la `memoria`, y usaremos para esto el metodo `load_memory_variables`, a diferencia del ejemplo anterior observaremos los mensajes

In [7]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='\nThe human greets the AI and the AI responds with a casual greeting.', additional_kwargs={})]}

Otra forma de obtener el `resumen`es usar el metodo `predict_new_summary`

In [10]:
messages = memory.chat_memory.messages

# Para que no se agregue mas informacion al resumen, enviaremos una cadena vacia
previous_summary=""

# Empleamos el metodo para obtener el resumen
memory.predict_new_summary(messages, previous_summary)

'\nThe human greets the AI, to which the AI responds.'

## Inicializando con mensajes o resumen
---

Si contamos con mensajes fuera de la clase, podemos inicializar facilmente con la clase `ChatMessageHistory`. 

Durante la carga, un resumen sera calculado.


Primero, hagamos un ejemplo cargando mediante `ChatMessageHistory`

In [12]:
history = ChatMessageHistory()

history.add_user_message("Hi")
history.add_ai_message("Hi there!")

Instanciemos la memoria

In [22]:
memory = ConversationSummaryMemory.from_messages(
    llm=llm,
    chat_memory=history,
    return_messages=True,
)

Veamos ahora el buffer

In [23]:
memory.buffer

'\nThe human greets the AI, to which the AI responds with a friendly greeting.'

Opcionalmente, puede acelerar la inicialización utilizando un resumen generado previamente y evitar volver a generar el resumen simplemente inicializándolo directamente.

In [24]:
memory = ConversationSummaryMemory(
    llm=llm,
    buffer="The human asks what the AI thinks of artificial intelligente. The AI thinks artificial intelligente is a force for good because it will help humans reach their full potential.",
    chat_memory=history,
    return_messages=True,
)

Veamos el resultado

In [25]:
memory.buffer

'The human asks what the AI thinks of artificial intelligente. The AI thinks artificial intelligente is a force for good because it will help humans reach their full potential.'

In [26]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human asks what the AI thinks of artificial intelligente. The AI thinks artificial intelligente is a force for good because it will help humans reach their full potential.', additional_kwargs={})]}

## Uso en un "chain"
---

Veamos un ejemplo usando este tipo de memoria en un "chain", usaremos la configuracion `verbose=True`, para que podamos observar el `prompt`

In [29]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory

# llm
llm = OpenAI(temperature=0)

# summary memory
memory = ConversationSummaryMemory(
    llm=llm,
)

# conversation chain
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [30]:
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?"

In [31]:
conversation_with_summary.predict(input="Tell me more about it!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human greets the AI and the AI responds that it is doing great and helping a customer with a technical issue.
Human: Tell me more about it!
AI:

> Finished chain.


" Sure! The customer is having trouble with their computer not connecting to the internet. I'm helping them troubleshoot the issue and figure out what the problem is. So far, I've been able to narrow it down to a few possible causes, but I'm still trying to determine the exact cause."

In [32]:
conversation_with_summary.predict(input="Very cool -- what is the scope of the project?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human greets the AI and the AI responds that it is doing great and helping a customer with a technical issue. The customer is having trouble with their computer not connecting to the internet and the AI is helping them troubleshoot the issue and figure out what the problem is. So far, the AI has been able to narrow it down to a few possible causes, but is still trying to determine the exact cause.
Human: Very cool -- what is the scope of the project?
AI:

> Finished chain.


" The scope of the project is to help the customer troubleshoot their computer's internet connection issue. I'm currently in the process of narrowing down the possible causes and trying to determine the exact cause of the issue."